# Model Error Analysis
1. Setup a project
2. Import labels with MAL
3. Correct the labels in the editor
4. Upload labels and predictions for MEA
------ 

## Environment Setup
* Install dependencies

In [ ]:
!pip install labelbox[data] == 3.0b0 \
             scikit-image \
             tensorflow

In [1]:
# Run these if running in a colab notebook
COLAB = "google.colab" in str(get_ipython())
if COLAB:
    !git clone https://github.com/Labelbox/labelbox-python.git
    !cd labelbox-python && git checkout mea-dev
    !mv labelbox-python/examples/model_assisted_labeling/*.py .

* Import libraries

In [2]:
from labelbox.schema.ontology import OntologyBuilder, Tool
from labelbox import Client, LabelingFrontend, MALPredictionImport
from image_model import predict, class_mappings, load_model
from io import BytesIO
from getpass import getpass
import uuid
import numpy as np
from PIL import Image
import requests
from collections import defaultdict
import ndjson
import os
from ndjson_utils import (
    create_boxes_ndjson, 
    create_polygon_ndjson, 
    create_mask_ndjson, 
    create_point_ndjson
)
from labelbox.data.metrics.iou import data_row_miou
from labelbox.data.serialization import NDJsonConverter
from labelbox.data.annotation_types import ScalarMetric, LabelList, Label, RasterData, Mask, Point, Rectangle, ObjectAnnotation

* Configure client

In [3]:
# If you don't want to give google access to drive you can skip this cell
# and manually set `API_KEY` below.

COLAB = "google.colab" in str(get_ipython())
if COLAB:
    !pip install colab-env -qU
    from colab_env import envvar_handler
    envvar_handler.envload()

API_KEY = os.environ.get("LABELBOX_API_KEY")
if not os.environ.get("LABELBOX_API_KEY"):
    API_KEY = getpass("Please enter your labelbox api key")
    if COLAB:
        envvar_handler.add_env("LABELBOX_API_KEY", API_KEY)

In [4]:
ENDPOINT = "https://api.labelbox.com/graphql"
client = Client(api_key=API_KEY, endpoint = ENDPOINT)

In [5]:
load_model()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from gs://cloud-tpu-checkpoints/mask-rcnn/1555659850/variables/variables


## Make Inferences

In [6]:
def get_annotations(boxes, classes, seg_masks):
    annotations = []
    for box, class_idx, seg in zip(boxes, classes, seg_masks):
        name = class_mappings[class_idx]
        value = None
        classifications = []
        if name in ['person', 'handbag']:
            value = Rectangle(
                start = Point(x = box[1], y = box[0]), end = Point(x = box[3], y = box[2])
            )
        elif name == 'car':
            value = Mask(mask = RasterData.from_2D_arr(arr = seg), color = (1,1,1))            
        if value is not None:
            annotations.append(
                ObjectAnnotation(
                    name = name,
                    value = value
                )
            ) 
    return annotations

### We can just start creating predictions whether or not we have a 
image_urls = ['https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg']
predictions = LabelList([])

for image_url in image_urls:
    image_data = RasterData(url = image_url)
    height, width = image_data.data.shape[:2]
    prediction = predict(np.array([image_data.im_bytes]), min_score=0.5, height=height, width = width)
    annotations = get_annotations(prediction['boxes'], prediction['class_indices'], prediction['seg_masks'])
    predictions.append(Label(
        data = image_data,
        annotations = annotations
    ))
    

## Project Setup

* Create project and attach dataset
* Automatically create an ontology from the predictions

In [7]:
project = client.create_project(name="mea_project")
dataset = client.create_dataset(name="mea_dataset")
editor = next(
    client.get_labeling_frontends(where=LabelingFrontend.name == 'editor'))
# Use the label list to build the ontology
project.setup(editor, predictions.get_ontology().asdict())
project.datasets.connect(dataset)
project.enable_model_assisted_labeling()

True

## Add Required Urls and Ids

In [8]:
signer = lambda _bytes: client.upload_data(content=_bytes, sign=True)
predictions.add_url_to_masks(signer) \
         .add_url_to_data(signer) \
         .assign_schema_ids(OntologyBuilder.from_project(project)) \
         .add_to_dataset(dataset, client.upload_data)

1it [00:02,  2.63s/it]
1it [00:00, 9986.44it/s]
1it [00:00, 10305.42it/s]


## MAL: Upload Predictions to a Project
* Pre-label image so that we can quickly create ground truth
* This step is not required for MEA.

In [9]:
upload_task = MALPredictionImport.create_from_objects(
    client, 
    project.uid, 
    f'mal-import-{uuid.uuid4()}', 
    list(NDJsonConverter.serialize(predictions))
)
upload_task.wait_until_done()
print(upload_task.state)

AnnotationImportState.FINISHED


In [10]:
upload_task.statuses[:5]

[{'uuid': 'a1ccac29-37c6-4e07-950d-5b80e5e6dc0d',
  'dataRow': {'id': 'ckros7pwl21gr0zu0dai6gg3p'},
  'status': 'SUCCESS'},
 {'uuid': 'ffb4229e-fccb-41fc-8cfd-002896a713ff',
  'dataRow': {'id': 'ckros7pwl21gr0zu0dai6gg3p'},
  'status': 'SUCCESS'},
 {'uuid': 'fe9046b6-6727-4b3e-b759-bd0215e11425',
  'dataRow': {'id': 'ckros7pwl21gr0zu0dai6gg3p'},
  'status': 'SUCCESS'},
 {'uuid': 'ac33a08d-8fb3-406a-b510-9fd43ac489f6',
  'dataRow': {'id': 'ckros7pwl21gr0zu0dai6gg3p'},
  'status': 'SUCCESS'},
 {'uuid': '2f09ae51-8408-47c7-8d77-d3eccc464ce5',
  'dataRow': {'id': 'ckros7pwl21gr0zu0dai6gg3p'},
  'status': 'SUCCESS'}]

## Label
* Create ground truth data for MEA
* Click on link below to label

In [11]:
print(f"https://app.labelbox.com/projects/{project.uid}")

https://app.labelbox.com/projects/ckros7k5g1qfe0y3v4ddihp0k


In [13]:
labels = project.label_generator().as_list()

## MEA
1. Create a model
    * Think of this as a model that you want to perform experiments on
2. Create a model run
    * Think of this as a single experiment for a particular model.
    * E.g. this model run is for an instance of a model with particular hyperparameters
3. Select the ground truth annotations for analysis
4. Compute metrics
4. Upload model predictions and metrics to labelbox

In [16]:
model = client.create_model(name = "test_model", ontology_id = project.ontology().uid)
model_run = model.create_model_run('test_run_1')

In [17]:
model_run.upsert_labels([label.uid for label in labels])

True

In [18]:
label_lookup = {label.data.uid : label for label in labels}

In [19]:
for pred in predictions:
    label = label_lookup[pred.data.uid]
    score = data_row_miou(label, pred)
    if score is None:
        continue
        
    pred.annotations.append(
        ScalarMetric(value = score)
    )
    

In [20]:
upload_task = model_run.add_predictions(f'mea-import-{uuid.uuid4()}', list(NDJsonConverter.serialize(predictions)))

In [21]:
upload_task.wait_until_done()
print(upload_task.state)

AnnotationImportState.FINISHED


In [22]:
upload_task.statuses

[{'uuid': 'ac0fb4ee-3177-413a-b0e0-75aa2611da27',
  'dataRow': {'id': 'ckros7pwl21gr0zu0dai6gg3p'},
  'status': 'SUCCESS'},
 {'uuid': '87f77120-fe38-4d3b-8162-214d910d47f9',
  'dataRow': {'id': 'ckros7pwl21gr0zu0dai6gg3p'},
  'status': 'SUCCESS'},
 {'uuid': 'e6103d58-7ebc-4f9f-b59e-3b7ee3a115ef',
  'dataRow': {'id': 'ckros7pwl21gr0zu0dai6gg3p'},
  'status': 'SUCCESS'},
 {'uuid': '056366b6-b581-4594-9758-cf64b76e7b93',
  'dataRow': {'id': 'ckros7pwl21gr0zu0dai6gg3p'},
  'status': 'SUCCESS'},
 {'uuid': 'f9171df5-511c-4aa2-9042-c15332d5194b',
  'dataRow': {'id': 'ckros7pwl21gr0zu0dai6gg3p'},
  'status': 'SUCCESS'},
 {'uuid': '0480cedf-0e9f-442c-b55d-8167ab57fd18',
  'dataRow': {'id': 'ckros7pwl21gr0zu0dai6gg3p'},
  'status': 'SUCCESS'},
 {'uuid': 'dac20fe5-c24e-4aec-94f7-8f037617daf8',
  'dataRow': {'id': 'ckros7pwl21gr0zu0dai6gg3p'},
  'status': 'SUCCESS'},
 {'uuid': 'd7802b8a-64cc-4fa7-9200-7cfbf62e89c9',
  'dataRow': {'id': 'ckros7pwl21gr0zu0dai6gg3p'},
  'status': 'SUCCESS'},
 {'uuid'

In [23]:
# View the uploaded predictions and labels (annotation group) in the web app
annotation_group = next(model_run.annotation_groups())
print(annotation_group.url)

https://app.labelbox.com/models/9beb00e5-0861-0eef-475e-1cc71824b158/9beb00e5-7a59-021e-d28b-c3319589f88e/AllDatarowsSlice/a8a345b7-d46f-44c8-bf77-237530ce10a1?view=carousel
